In [1]:
import pandas as pd
import numpy as np
import csv
from os import listdir

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [ ]:
# Dit is de partij_df met het aantal stemmen per partij gebaseerd op de peilingen.
partij_df_peiling = pd.DataFrame.from_csv('partij_df_peiling.csv', sep=';')
partij_df_peiling

In [ ]:
# Dit is de partij_df met het aantal stemmen per partij gebaseerd op de einduitslag.
partij_df_uitslag = pd.DataFrame.from_csv('partij_df_uitslag.csv', sep=';')
partij_df_uitslag

In [ ]:
# stemmen op vrouwen wat je zou verwachten op basis van de peiling.

stemmen_van_vrouwen_peiling = partij_df_peiling[['Partij','# stemmen partij', 'Peiling', 'Max_Vrouwen_Volgens_Peilingen']].drop_duplicates()
stemmen_van_vrouwen_peiling['# vrouwen stemmen'] = (0.5*stemmen_van_vrouwen_peiling['# stemmen partij']).astype(int)
stemmen_van_vrouwen_peiling

In [ ]:
# stemmen op vrouwen gebaseerd op de einduitlag.

stemmen_van_vrouwen_uitslag = partij_df_uitslag[['Partij','# stemmen partij', 'Peiling', 'Aantal Vrouwen', 'Vrouw_stem_percentage',
                                                 'Max_Vrouwen_Volgens_Peilingen']].drop_duplicates()
stemmen_van_vrouwen_uitslag['Vrouw_stem_percentage'] = stemmen_van_vrouwen_uitslag['Vrouw_stem_percentage']/100
stemmen_van_vrouwen_uitslag['# vrouwen stemmen'] = (stemmen_van_vrouwen_uitslag['Vrouw_stem_percentage']*stemmen_van_vrouwen_uitslag['# stemmen partij']).astype(int)
stemmen_van_vrouwen_uitslag
                                                 

In [ ]:
# Het aantal stemmen pertop N kanidaat op basis van de peiling.

partijen = pd.unique(partij_df_peiling.Partij.ravel())
max_vrouwen_dict = {}
for partij in partijen:
    if partij != 'SGP':
        loop_stemmen_partij_df = stemmen_van_vrouwen_peiling[stemmen_van_vrouwen_peiling['Partij'] == partij]
        max_vrouwen_dict[partij] = loop_stemmen_partij_df['# vrouwen stemmen'].item()/loop_stemmen_partij_df['Max_Vrouwen_Volgens_Peilingen'].item()


# hieronder wordt uiteindelijk een df aangemaakt die per partij aangeeft hoeveel stemmen er zijn toewezen per top N kandidaat.    
stemmen_op_vrouwen_peiling = pd.DataFrame.from_dict(max_vrouwen_dict, orient='index', dtype=None)
stemmen_op_vrouwen_peiling = stemmen_op_vrouwen_peiling.reset_index()
stemmen_op_vrouwen_peiling.columns = ['Partij' , 'stemmen per vrouw']
stemmen_op_vrouwen_peiling['stemmen per vrouw'] = stemmen_op_vrouwen_peiling['stemmen per vrouw'].astype(int)
stemmen_op_vrouwen_peiling#.to_csv('stemmen_op_vrouwen_topN_peiling.csv', sep=',')

In [ ]:
# Het aantal stemmen per top N kanidaat gebaseerd op de einduitslag. 

max_vrouwen_dict = {}
for partij in partijen:
    if partij != 'SGP':
        loop_stemmen_partij_df = stemmen_van_vrouwen_uitslag[stemmen_van_vrouwen_uitslag['Partij'] == partij]
        max_vrouwen_dict[partij] = loop_stemmen_partij_df['# vrouwen stemmen'].item()/loop_stemmen_partij_df['Max_Vrouwen_Volgens_Peilingen'].item()
        
        
# hieronder wordt uiteindelijk een df aangemaakt die per partij aangeeft hoeveel stemmen er zijn toewezen per top N kandidaat.    
stemmen_op_vrouwen_uitslag = pd.DataFrame.from_dict(max_vrouwen_dict, orient='index', dtype=None)
stemmen_op_vrouwen_uitslag = stemmen_op_vrouwen_uitslag.reset_index()
stemmen_op_vrouwen_uitslag.columns = ['Partij' , 'stemmen per vrouw']
stemmen_op_vrouwen_uitslag['stemmen per vrouw'] = stemmen_op_vrouwen_uitslag['stemmen per vrouw'].astype(int)
stemmen_op_vrouwen_uitslag#.to_csv('stemmen_op_vrouwen_topN_uitslag.csv', sep=',')

In [ ]:
# Een copy van partij_df_peiling wordt gemaakt. In de copy worden de het aantal stemmen van stemmen_op_vrouwen_uitslag gegeven aan de top kandidaten per partij.

partij_df_copy = partij_df_peiling.copy() # copy van partij_df_peiling om values van aantal stemmen kandidaat(vrouw in dit geval) te veranderen.

for partij in partijen:
    if partij != 'SGP':
        # aantal vrouwelijk kandidaten per partij dat de stemmen toegewezen krijgen (top N).
        aantal_vrouwen_per_partij = pd.unique(stemmen_van_vrouwen_peiling[stemmen_van_vrouwen_peiling['Partij'] == partij]['Max_Vrouwen_Volgens_Peilingen']).item()
        # alle vrouwelijke kandidaten per partij.
        vrouwen_per_partij = partij_df_peiling[(partij_df_peiling['Partij'] == partij)&(partij_df_peiling['Geslacht']=='V')]['Kandidaat'].values 
        
        for i in range(int(aantal_vrouwen_per_partij)):
            # aantal_stemmen_vrouwen_i is het aantal stemmen top N vrouwelijke kandidaten krijgen.
            aantal_stemmen_vrouw_i = stemmen_op_vrouwen_uitslag[stemmen_op_vrouwen_uitslag['Partij'] == partij]['stemmen per vrouw'].item() 
            # hier krijgen de top N vrouwelijke kandidaten de stemmen in partij_df_copy.
            partij_df_copy.loc[partij_df_copy['Kandidaat'] == vrouwen_per_partij[i], '# stemmen kandidaat'] = aantal_stemmen_vrouw_i

partij_df_copy

In [ ]:
# Het samenstellen van de Tweede Kamer na uitvoering van strategie 1.

tweede_kamer_lijst = []
for partij in partijen:
    # Aantal zetels wordt verdeeld aan de hand van de einduitslag. 'Zetels' staat voor het daadwerkelijke aantal zetels dat een partij ontving.
    aantal_zetels = pd.unique(partij_df_uitslag[(partij_df_peiling['Partij'] == partij)]['Zetels'].ravel())[0]
    # aantal_boven_voorkeursdrempel is het aantal kandidaten boven de voorkeursdrempel (=15708).
    aantal_boven_voorkeursdrempel = len(partij_df_copy[(partij_df_peiling['Partij'] == partij)&(partij_df_copy['# stemmen kandidaat'] > 15708)]) 
    verschil = aantal_zetels - aantal_boven_voorkeursdrempel
    
    # Aparte df per partij om te sorteren op '# stemmen kandidaat' waarbij alles wordt geselecteerd van 0 tot het minimum van
    # aantal_zetels en aantal_boven_voorkeurs_drempel(welke van de 2 het minste is dus).
    per_partij_df = partij_df_copy[partij_df_copy['Partij'] == partij]
    eerste_deel = per_partij_df.sort_values(by=['# stemmen kandidaat'], ascending=False)[0:min(aantal_zetels,aantal_boven_voorkeursdrempel)]
    # per partij het eerste deel toegevoegd.
    tweede_kamer_lijst.append(eerste_deel)
    
    # als er meer zetels volgens de peilingen zijn dan er boven de voorkeursdrempel zijn gebracht(verschil) dan worden 
    # de rest van de kandidaten per partij aan de tweede kamer toegevoegd(dit is wanneer de peilingen meer zetels aangeven dan er
    # vrouwen in die partij op de kandiatenlijst staan)
    if verschil > 0:
        tweede_deel = per_partij_df[~per_partij_df['Kandidaat'].isin(eerste_deel['Kandidaat'])].dropna()[0:verschil]
        tweede_kamer_lijst.append(tweede_deel)

        
# hieronder wordt uiteindelijk een df aangemaakt op basis van tweede_kamer_lijst.   
tweede_kamer_strategie_topN = pd.concat(tweede_kamer_lijst)        
tweede_kamer_strategie_topN = tweede_kamer_strategie_topN[['Partij','Kandidaat', 'Geslacht', '# stemmen kandidaat']].reset_index(drop=True)
pd.set_option('display.max_rows', 150)
tweede_kamer_strategie_topN

In [10]:
len(tweede_kamer_strategie_topN[tweede_kamer_strategie_topN['Geslacht'] == 'V'])

121